In [28]:
import pandas as pd
import spacy
from concurrent.futures import ThreadPoolExecutor
# Загрузка данных из Excel
df = pd.read_excel("ner_data_train.xlsx")

In [29]:
# Описание тегов
tag_descriptions = {
    "O": "Неопределенный",
    "B-Дата": "Дата",
    "B-бренд": "Бренд",
    "B-вид спорта": "Вид спорта",
    "B-видеоигра": "Видеоигра",
    "B-команда": "Команда",
    "B-лига": "Лига",
    "B-локация": "Локация",
    "B-модель": "Модель",
    "B-название проекта": "Название проекта",
    "B-организация": "Организация",
    "B-персона": "Персона",
    "B-сезон": "Сезон",
    "B-серия": "Серия",
    "I-Дата": "Дата (продолжение)",
    "I-бренд": "Бренд (продолжение)",
    "I-вид спорта": "Вид спорта (продолжение)",
    "I-видеоигра": "Видеоигра (продолжение)",
    "I-команда": "Команда (продолжение)",
    "I-лига": "Лига (продолжение)",
    "I-локация": "Локация (продолжение)",
    "I-модель": "Модель (продолжение)",
    "I-название проекта": "Название проекта (продолжение)",
    "I-организация": "Организация (продолжение)",
    "I-персона": "Персона (продолжение)",
    "I-сезон": "Сезон (продолжение)",
    "I-серия": "Серия (продолжение)",
}

In [30]:

# Массив тегов, которые нужно искать
tags_to_extract = list(tag_descriptions.keys())

# Загрузка spaCy модели для русского языка
nlp = spacy.load("ru_core_news_sm")

# Функция для извлечения тегов
def extract_tags(text):
    doc = nlp(text)
    tags = []
    current_label = ""
    for token in doc:
        label = tag_descriptions.get(token.ent_iob_ + "-" + token.ent_type_, "O")
        if label != current_label:
            if current_label != "O":
                tags.append(current_label)
            current_label = label
        tags.append(token.text)
    if current_label != "O":
        tags.append(current_label)
    return tags

# Функция для обработки нескольких строк данных параллельно
def process_batch(batch):
    result = []
    for text in batch:
        tags = extract_tags(text)
        result.append(tags)
    return result

# Разделение данных на пакеты для параллельной обработки
batch_size = 10
batches = [df["video_info"][i:i+batch_size] for i in range(0, len(df), batch_size)]

# Использование ThreadPoolExecutor для параллельной обработки
with ThreadPoolExecutor(max_workers=4) as executor:
    processed_batches = list(executor.map(process_batch, batches))

# Объединение результатов
tags_result = [tag for batch in processed_batches for tag in batch]

# Добавление результатов в DataFrame
df["tags"] = tags_result

# Сохранение результата в новом Excel файле
df.to_excel("output_processed_with_tags.xlsx", index=False)

# Вывод обработанных данных
print(df)


                                             video_info  \
0     <НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...   
1     <НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...   
2     <НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...   
3     <НАЗВАНИЕ:> Довоенная немецкая кирха в Калинин...   
4     <НАЗВАНИЕ:> "Спартаку" помогли судьи? Локомоти...   
...                                                 ...   
6417  <НАЗВАНИЕ:> ЗАПРЕЩЕННЫЙ УДАР! Полный бой Вадим...   
6418  <НАЗВАНИЕ:> Как создаются медали: оптические и...   
6419  <НАЗВАНИЕ:> Ремонт ломовоза камаз, монтаж гидр...   
6420  <НАЗВАНИЕ:> Голодные игры: Баллада о змеях и п...   
6421  <НАЗВАНИЕ:> Экскаватор на Дровах! Часть 15. За...   

                                               entities  \
0     {"label":"локация"\,"offset":26\,"length":6\,"...   
1     {"label":"организация"\,"offset":196\,"length"...   
2     {"label":"название проекта"\,"offset":12\,"len...   
3     {"label":"не найдено"\,"offset":162\,"length":...